In [27]:
import pandas as pd
import keras
from keras import layers
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error
import tensorflow as tf
import matplotlib

## Add this line for reproducibility reasons (to have the same random seed in all keras layers)
tf.keras.utils.set_random_seed(100)

In [28]:
def visualize_loss(history, title,save_params):
    """
    Visualize the training procedure (Identify any case of over or under fitting)
    :param history: history of the training procedure
    :param title: title of the image produced 
    """
    
    # todo: add a flag for saving or not the figure
    
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    if save_params['save_figure']:
        plt.savefig(save_params['path']+'loss.png')
    plt.show()
    
    

In [29]:
def evaluate_reconstruction(test, pred,save_params):
    """
    Evaluate the reconstruction values of the test set.
    Creates scatter plots with the true and the predicted values of the autoencoder. 
    :param test: test set of the data
    :param pred: predictions of the test set
    
    """
    
    fig, ax = plt.subplots(1, 3, figsize=(12, 4))
    ax[0].scatter(test['T'], pred[:, 0])
    ax[0].set_title('Temperature')
    ax[0].set_xlabel('True values')
    ax[0].set_ylabel('Reconstructed values')
    #######################
    ax[1].scatter(test['P'], pred[:, 1])
    ax[1].set_title('Pressure')
    ax[1].set_xlabel('True values')
    ax[1].set_ylabel('Reconstructed values')
    ###########
    ax[2].scatter(test['D'], pred[:, 2])
    ax[2].set_title('Diffusivity')
    ax[2].set_xlabel('True values')
    ax[2].set_ylabel('Reconstructed values')
    if save_params['save_figure']:
        plt.savefig(save_params['path']+'reconstruction.png')
    plt.show()
    # fig.clf()

In [30]:
def error(dfin, dfout):
    """
    Calculate the error between the predicted and actual
    """
    t_error = (dfin[:, 0] - dfout[:, 0]) ** 2
    p_error = (dfin[:, 1] - dfout[:, 1]) ** 2
    d_error= (dfin[:, 2] - dfout[:, 2]) ** 2
    error_all = t_error + p_error + d_error
    error_df = pd.DataFrame([t_error, p_error, d_error, error_all]).transpose()
    
    return error_df

In [31]:
def plot_sensitivity(_df_results, name, _save_params):
    """
    Creates a 3D plot with all Diffusivity values over temperature and pressure. The experimental results are also displayed in this plot. 
    
    """    
    _md = pd.read_csv('../data/CO2_clean.csv', usecols=['Index', 'T', 'P', 'D'])
    
    
    
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    
    ax.scatter(_df_results['T'], _df_results['P'], _df_results['D'],alpha=0.2)
    ax.scatter(_md['T'], _md['P'], _md['D'],alpha=0.5, s=80, marker='^')
    ax.legend(['Predictions', 'MD'], loc='upper left')
    
    ax.set_xlabel('Temperature\n(K)')
    ax.set_ylabel('Pressure\n(MPa)')
    ax.set_zlabel('Diffusivity\n($10^{-9} m^2/s$)')
    
    if _save_params['save_figure']:
        plt.savefig(_save_params['save_path'] + '/{}_sensitivity.png'.format(name))
    plt.show()

In [32]:
def export_results_autoencoder(_model, _scaler):
    """
    Exports the trained autoencoder 
    """

    diffusivities = np.arange(0, 25, 25 / 1000)  # create a vector with small step to locate the minimum error

    _results_df = pd.DataFrame()
    for i in range(275, 625, 10):
        temperatures = [i for d in range(len(diffusivities))]
        for j in range(1, 1000, 10):
            Pressure = j / 10

            pressures = [Pressure for i in range(len(temperatures))]
            temp_max = 625
            # temp_max = 369.9 * Pressure ** (0.089)
            temp = {'T': temperatures, 'P': pressures, 'D': diffusivities}
            _df_for_test = pd.DataFrame(temp)
            _df_for_test = _df_for_test[_df_for_test['T'] < temp_max]
            try:
                _temp_df_scaled = _scaler.transform(_df_for_test)
                _results = _model.predict(_temp_df_scaled)
                # use the error function to get the error values for all T and P pairs
                _error_df = error(_temp_df_scaled, _results)
                # find the diffusivity value for the specific P and T with minimum error
                _min_error = _error_df[[3]].idxmin()
                _best_value = _df_for_test.iloc[_min_error]
                _results_df = pd.concat([_results_df, _best_value], ignore_index=True)


            except:
                continue



    return _results_df

In [34]:
def plot_boxplot(_df,_save_params):
    plt.boxplot(_df[0])
    plt.ylabel('MSE')
    plt.title('Autoencoder MSE')
    plt.xticks([1],['{} runs'.format(len(_df[0]))])
    if _save_params['save_figure']:
        plt.savefig(_save_params['path'] + 'boxplot_Autoencoder.png', dpi=300) 
    plt.show()

In [ ]:

if __name__ == "__main__":
    
    %matplotlib notebook
    save_params ={'path': '../results/',
                  'save_figure' : False}
    
    # load the data
    df = pd.read_csv('../data/CO2_clean.csv', usecols=['Index', 'T', 'P', 'D'])
    X = df[['T', 'P', 'D']]
    y = df['D']

    # Define dataframes 
    results_auto = {}
    predictions_auto = pd.DataFrame()
    
    ###########################
    ####### AUTOENCODER #######
    ###########################
    
    ## Set callbacks ##
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='auto')
    reduce_lron_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto')

    ##  Define the Architecture ##
    input_layer = keras.Input(shape=(3,), name='input_layer')
    encoder_layer_1 = layers.Dense(44, activation='elu', name='encoder_layer_1')(input_layer)
    encoder_layer_2 = layers.Dense(72, activation='elu', name='encoder_layer_2')(encoder_layer_1)
    latent = layers.Dense(2, activation='elu', name='latent_space')(encoder_layer_2)
    decoder_layer_1 = layers.Dense(72, activation='elu', name='decoder_layer_1')(latent)
    decoder_layer_2 = layers.Dense(44, activation='elu', name='decoder_layer_2')(decoder_layer_1)
    output_layer = layers.Dense(3, activation='elu', name='output_layer')(decoder_layer_2)


    for seed in range(100):
        
        ## Clear keras session before each train
        keras.backend.clear_session()
        
        ## Split and scale the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=seed)         
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        ## Create and train the AUTOENCODER ##
        autoencoder = keras.Model(input_layer, output_layer)
        autoencoder.compile(optimizer='adam', loss='mse')
        history = autoencoder.fit(X_train_scaled, X_train_scaled,
                                  verbose=0,
                                  epochs=1000,
                                  batch_size=32,
                                  shuffle=False,
                                  validation_data=(X_train_scaled, X_train_scaled),
                                  callbacks=[reduce_lron_plateau, early_stopping])

        # Check training progress for some seeds
        if seed in [0,50,99]:
            # visualize loss #
            visualize_loss(history, 'Autoencoder loss',save_params)
            # Check reconstruction #
            autoencoder_reconstruction = autoencoder.predict(X_test_scaled)
            autoencoder_reconstruction = scaler.inverse_transform(autoencoder_reconstruction)
            evaluate_reconstruction(X_test, autoencoder_reconstruction,save_params)

        ## Find the D's using the Autoencoder ## 
        diffusivity_values = np.arange(0, 25, 25 / 1000)  # create a vector with small step to locate the minimum error
        autoencoder_test = pd.DataFrame()
        
        for index, row in X_test.iterrows():
            
            ## Create Pressure and Temperature vectors for each D
            pressures = [row['P'] for pressure in range(len(diffusivity_values))]
            temperatures = [row['T'] for temperature in range(len(diffusivity_values))]
            
            ## Create and scale a Dataframe from the P, T and D vectors 
            generated_df = pd.DataFrame.from_dict({'T': temperatures, 'P': pressures, 'D': diffusivity_values})
            temp_df_scaled = scaler.transform(generated_df)
            
            # get the autoencoder predictions
            results = autoencoder.predict(temp_df_scaled)
            # use the error function to get the error values for all T and P pairs
            error_df = error(temp_df_scaled, results)
            # find the diffusivity value for the specific P and T with minimum error
            min_error = error_df[[3]].idxmin()
            best_value = generated_df.iloc[min_error]
            autoencoder_test = pd.concat((autoencoder_test, best_value), ignore_index=True)
      
    

        autoencoder_mse = mean_squared_error(autoencoder_test['D'], X_test['D'])
        
        y_auto_test = X_test['D'].reset_index()
        temp_auto = pd.DataFrame({'y_pred':autoencoder_test['D'], 'y_test':y_auto_test['D']})
        
        
        predictions_auto = pd.concat([predictions_auto,temp_auto], ignore_index=True)
        
        results_auto[seed] = autoencoder_mse
        # print()
    results_df = pd.DataFrame(results_auto, index=[0])
    plot_boxplot(results_df,save_params)
    
    print("Done!")


Epoch 30: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 43: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 53: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 63: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 73: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 83: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 93: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 103: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 113: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 123: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 133: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 143: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.

Epoch 153: ReduceLROnPlateau reducing learning rate to 1

<IPython.core.display.Javascript object>

32/32 [==============================] - 0s 627us/step

Epoch 15: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 33: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 43: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 53: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 63: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 73: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 83: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 93: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 103: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 113: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 123: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 133: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.

Ep

<IPython.core.display.Javascript object>

32/32 [==============================] - 0s 139us/step

Epoch 11: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 21: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.

Epoch 31: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.

Epoch 41: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.

Epoch 51: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.

Epoch 61: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.

Epoch 71: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.

Epoch 81: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.

Epoch 91: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.

Epoch 101: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.

Epoch 111: ReduceLROnPlateau reducing learning rate to 4.882812731921149e-07.

Epoch 121: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.

Epo

In [ ]:
###########################
####### AUTOENCODER #######
###########################
df = pd.read_csv('../data/CO2_clean.csv', usecols=['Index', 'T', 'P', 'D'])
X = df[['T', 'P', 'D']]
y = df['D']
## Set callbacks ##
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='auto')
reduce_lron_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, mode='auto')

##  Define the Architecture ##
input_layer = keras.Input(shape=(3,), name='input_layer')
encoder_layer_1 = layers.Dense(44, activation='elu', name='encoder_layer_1')(input_layer)
encoder_layer_2 = layers.Dense(72, activation='elu', name='encoder_layer_2')(encoder_layer_1)
latent = layers.Dense(2, activation='elu', name='latent_space')(encoder_layer_2)
decoder_layer_1 = layers.Dense(72, activation='elu', name='decoder_layer_1')(latent)
decoder_layer_2 = layers.Dense(44, activation='elu', name='decoder_layer_2')(decoder_layer_1)
output_layer = layers.Dense(3, activation='elu', name='output_layer')(decoder_layer_2)

scaler_full = MinMaxScaler()
X_scaled = scaler_full.fit_transform(X)
autoencoder_full = keras.Model(input_layer, output_layer)
autoencoder_full.compile(optimizer='adam', loss='mse')
history_full = autoencoder_full.fit(X_scaled, X_scaled,
                               verbose=0,
                               epochs=1000,
                               batch_size=32,
                               shuffle=False,
                               validation_data=(X_scaled, X_scaled),
                               callbacks=[reduce_lron_plateau, early_stopping])

df_auto_results_full = export_results_autoencoder(autoencoder_full, scaler_full)
plot_sensitivity(df_auto_results_full,'Autoencoder',save_params)